In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [39]:
#!/usr/bin/env python
# coding: utf-8
from datasets import load_dataset
from transformers import BertTokenizerFast

dataset=load_dataset("squad")
tokenizer=BertTokenizerFast.from_pretrained("bert-base-uncased")
train_ds=dataset["train"].select(range(600))

In [40]:
example = dataset['train'][0]
inputs = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        padding="max_length",
        max_length=384,
        return_offsets_mapping=True,
    )

inputs.keys()

offset_mapping = inputs["offset_mapping"]  # of that example text str
start_char = example["answers"]["answer_start"][0]
end_char = start_char + len(example["answers"]["text"][0])

sequence_ids = inputs.sequence_ids()


def preprocess(example):
    inputs = tokenizer(
        example["question"],
        example["context"],
        truncation=True,
        padding="max_length",
        max_length=384,
        return_offsets_mapping=True,
    )

    offset_mapping = inputs['offset_mapping']
    attention_mask = inputs['attention_mask']
    start_char = example['answers']["answer_start"][0]
    end_char = start_char + len(example['answers']['text'][0])
    sequence_ids = inputs.sequence_ids()

    inputs["start_positions"] = 0
    inputs["end_positions"] = 0

    if len(example['answers']['answer_start']) > 0:
        start_char = example['answers']["answer_start"][0]
        end_char = start_char + len(example['answers']['text'][0])

        for i , (offset, sequence_id) in enumerate(zip(offset_mapping,sequence_ids)):
            if sequence_id == 1: # because its for the context. Question and answers have it to be 0
                if offset[0] <= start_char < offset[1]:
                    inputs["start_positions"] = i
                if offset[0] < end_char <= offset[1]:
                    inputs["end_positions"] = i

    return {
        "input_ids": inputs["input_ids"],
        "attention_mask": inputs["attention_mask"],
        "start_positions": inputs["start_positions"],
        "end_positions": inputs["end_positions"]
    }


tokenized_data = train_ds.map(preprocess, remove_columns=train_ds.column_names)

# In[84]:
#tokenized_data[0]['end_positions']

In [50]:
import torch 
from torch.utils.data import DataLoader
from transformers import BertForQuestionAnswering
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5) # SGD m0.9, AdamW 3e-5,
#criterion = torch.nn.CrossEntropyLoss()

In [47]:
tokenized_data.set_format(type="torch", columns=["input_ids", "attention_mask", "start_positions", "end_positions"])
train_loader = DataLoader(tokenized_data, batch_size=16, shuffle=True)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [51]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased").to(device)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [52]:
torch.cuda.empty_cache() #empty cache
#set the training mode
model.train()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, 

In [53]:
for epoch in range(15):  #iterationsss
    for batch in train_loader:
        optimizer.zero_grad()
        #running_loss = 0
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        outputs = model(input_ids=input_ids,  attention_mask=attention_mask,   
                        start_positions=start_positions,    
                        end_positions=end_positions)
        loss = outputs.loss
        #logits = outputs.logits
        #loss = criterion(logits, labels)       
        loss.backward()
        optimizer.step()        
        #running_loss += loss.item()
    print(f"Epoch {epoch+1}. Loss:{loss:.4f}")

Epoch 1. Loss:5.8769
Epoch 2. Loss:5.8774
Epoch 3. Loss:5.7862
Epoch 4. Loss:5.9499
Epoch 5. Loss:5.9493
Epoch 6. Loss:6.0097
Epoch 7. Loss:5.9521
Epoch 8. Loss:5.9715
Epoch 9. Loss:6.0125


KeyboardInterrupt: 

In [ ]:
output_dir = "./my_trained_Bert-Squad" # You can choose any path you like

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 2. Save the model's weights and configuration
# This saves the 'state_dict' (model weights) and 'config.json' (model architecture and settings)
model.save_pretrained(output_dir)

In [ ]:
test_data = load_dataset("squad")["validation"].select(range(50))
tokenized_test = test_data.map(preprocess, remove_columns=test_data.column_names)
tokenized_test.set_format(type="torch", columns=["input_ids", "attention_mask", "start_positions", "end_positions"])
test_loader = DataLoader(tokenized_test, batch_size=8)

In [ ]:
#EVAL
model.eval()

predictions = []
references = []
for i, batch in enumerate(test_loader):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    for j in range(input_ids.shape[0]):
       start_idx = torch.argmax(start_logits[j])
       end_idx = torch.argmax(end_logits[j])
       
       if(start_idx>end_idx):
           start_idx, end_idx = end_idx, start_idx
       end_idx +=1
       print(start_idx, end_idx)
       input_id = input_ids[j]
       tokens = tokenizer.convert_ids_to_tokens(input_id[start_idx:end_idx])
       answer = tokenizer.convert_tokens_to_string(tokens)
       #print("tokens: ", tokens)
       print("answer obtained: ", answer)
       print("original ans: ", test_data[i * 8 + j]["answers"]["text"])  # match by batch index)
    break

In [ ]:
for i, batch in enumerate(test_loader):
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # print("start_logits",start_logits)
    # print("end logits",end_logits)
    for j in range(input_ids.shape[0]):
        start_idx = torch.argmax(start_logits[j])
        end_idx = torch.argmax(end_logits[j])

        if(start_idx>end_idx):
           start_idx, end_idx = end_idx, start_idx

        end_idx +=1
        input_id = input_ids[j]
        tokens = tokenizer.convert_ids_to_tokens(input_id[start_idx:end_idx])
        answer = tokenizer.convert_tokens_to_string(tokens)

        original = test_data[i * 8 + j]  # match by batch index
        predictions.append({"id": original["answers"]['text'], "prediction_text": answer})
        #references.append({"id": original["id"], "answers": original["answers"]})


#
predictions